In [1]:
from vpython import *
import ikpy.chain
import xml.etree.ElementTree as ET
import numpy as np
my_chain = ikpy.chain.Chain.from_urdf_file("arm_urdf.urdf",active_links_mask=[False, True, True, True, True, True])

<IPython.core.display.Javascript object>

In [136]:
def axisAngleRotationMatrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    a = cos(theta / 2.0)
    b, c, d = -axis * sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
class Link:
    def __init__(self, pos=np.zeros(3), size=np.ones(3), axis=None, jointOrigin=None):
        self.pos = pos
        temp = self.pos[2]
        self.pos[2] = self.pos[1]
        self.pos[1] = temp
        self.size = size
        temp = self.size[2]
        self.size[2] = self.size[1]
        self.size[1] = temp
        if np.any(axis == None):
            self.jointAxis = np.array([0., 1., 0.])
        else:
            self.jointAxis = axis
            temp = self.jointAxis[2]
            self.jointAxis[2] = self.jointAxis[1]
            self.jointAxis[1] = temp
        if np.any(jointOrigin == None):
            self.jointOrigin = self.pos.copy()
        else:
            self.jointOrigin = jointOrigin
            temp = self.jointOrigin[2]
            self.jointOrigin[2] = self.jointOrigin[1]
            self.jointOrigin[1] = temp
        self.angle = 0
        self.axis = np.array([1., 0., 0.])
        self.up = np.array([0., 1., 0.])
        self.visual = box(pos=vec(pos[0], pos[1], pos[2]), size=vec(size[0], size[1], size[2]), color=vec(0.5, 0.5, 1))
    def update(self, pos, angle):
        self.pos = pos
        self.visual.pos = vec(self.pos[0], self.pos[1], self.pos[2])
        self.axis = np.dot(axisAngleRotationMatrix(self.jointAxis, angle - self.angle), self.axis)
        self.up = np.dot(axisAngleRotationMatrix(self.jointAxis, angle - self.angle), self.up)
        self.visual.up = vec(*self.up)
        self.visual.axis = vec(*self.axis)
        self.visual.size = vec(*self.size)
        self.angle = angle
        
class Robot:
    def __init__(self, links):
        self.links = links
    def setAngle(self, linkIdx, angle):
        if linkIdx == 0:
            print("BASE LINK CANT MOVE")
            return
        # link = self.links[linkIdx]
        # dAngle = angle - link.angle
        # link.angle = angle
        # rotMat = axisAngleRotationMatrix(link.jointAxis, dAngle)
        # link.axis = np.dot(rotMat, link.axis)
        # prevEndPoint = link.jointOrigin + np.max(link.size)*link.up
        # link.up = np.dot(rotMat, link.up)
        # endPoint = link.jointOrigin + np.max(link.size)*link.up
        # self.links[linkIdx+1].jointOrigin += (endPoint-prevEndPoint)
        # link.pos = link.jointOrigin + np.max(link.size)*link.up*0.5
        # link.visual.pos = vec(*link.pos)
        # link.visual.axis = vec(*link.axis)
        # link.visual.up = vec(*link.up)
        # link.visual.size = vec(*link.size)

        for i in range(linkIdx, len(self.links)):
            
            link = self.links[i]
            if i == 1:
                moveAxis = link.axis
            else:
                moveAxis = link.up
            if i == linkIdx:
                dAngle = angle - link.angle
                link.angle = angle
                rotMat = axisAngleRotationMatrix(link.jointAxis, dAngle)
                prevEndPoint = link.jointOrigin + np.max(link.size)*moveAxis
            link.jointAxis = np.dot(rotMat, link.jointAxis)
            link.axis = np.dot(rotMat, link.axis)
            link.up = np.dot(rotMat, link.up)
            if i == 1:
                moveAxis = link.axis
            else:
                moveAxis = link.up
            endPoint = link.jointOrigin + np.max(link.size)*moveAxis
            if i < len(self.links)-1:
                temp = self.links[i+1].jointOrigin + np.max(self.links[i+1].size)*self.links[i+1].up
                # self.links[i+1].jointOrigin += (endPoint-prevEndPoint)
                # EXTREMELY HARD CODED DONT DO THIS LOL
                
                if i == 2:
                    self.links[i+1].jointOrigin = endPoint-0.07*link.axis
                elif i == 3:
                    self.links[i+1].jointOrigin = endPoint+0.07*link.axis
                elif i == 1:
                    self.links[i+1].jointOrigin += (endPoint-prevEndPoint)
                elif i == 4:
                    self.links[i+1].jointOrigin = endPoint-0.01*link.up
                # elif i == 4:
                #     self.links[i+1].jointOrigin = endPoint-0.07*link.axis
                prevEndPoint = temp
            if i == 5:
                link.pos = link.jointOrigin
            else:
                link.pos = link.jointOrigin + np.max(link.size)*moveAxis*0.5
            link.visual.pos = vec(*link.pos)
            link.visual.axis = vec(*link.axis)
            link.visual.up = vec(*link.up)
            link.visual.size = vec(*link.size)

        

In [145]:
scene = canvas()


# Parse the URDF file
tree = ET.parse('arm_urdf.urdf')
root = tree.getroot()

# Access robot properties
robot_name = root.attrib.get('name')
# print(robot_name)
summedPos = np.zeros(3)
joints = {}
for joint in root.findall('joint'):
    parent = joint.findall('parent')[0].attrib['link']
    child = joint.findall('child')[0].attrib['link']
    origin = np.array([float(i) for i in joint.findall('origin')[0].attrib['xyz'].split(" ")])
    axis = np.array([float(i) for i in joint.findall('axis')[0].attrib['xyz'].split(" ")])
    # print("JOINT", parent, child, origin, axis)
    joints[child] = [origin, axis]

links = []
for link in root.findall('link'):
    name = link.attrib['name']
    # print(name)
    visual = link.findall('visual')[0]
    origin = visual.findall('origin')[0]
    geom = visual.findall('geometry')[0]
    if len(geom.findall('box'))>0:
        # print("BOX")
        geom = geom.findall('box')[0]
        size = np.array([float(i) for i in geom.attrib['size'].split(" ")])
    else:
        # print("CYLINDER")
        geom = geom.findall('cylinder')[0]
        radius = float(geom.attrib['radius'])
        length = float(geom.attrib['length'])
        size = np.array([radius*2, radius*2, length])
    origin = np.array([float(i) for i in origin.attrib['xyz'].split(" ")])
    
    axis = None
    jointOrigin = None
    if name in joints:
        jointOrigin = joints[name][0]
        axis = joints[name][1]
        summedPos += jointOrigin
    origin += summedPos
    
    # print("Pos", origin)
    # print("Size", size)
    links.append(Link(origin, size, axis, summedPos.copy()))
# links[3].visual.color=vec(1, 0, 0)
robot = Robot(links)
# robot.setAngle(1, 3.14159/4)
# robot.setAngle(2, 3.14159/4)
# robot.setAngle(3, 3.14159/4)
# robot.setAngle(4, 3.14159/4)
# robot.setAngle(5, 3.14159/4)

t = 0
dt = 0.01
while t < 10:
    robot.setAngle(1, sin(t)*2*3.14159)
    robot.setAngle(2, sin(t)*0.25*3.14159)
    robot.setAngle(3, sin(t)*0.25*3.14159)
    robot.setAngle(4, sin(t)*0.25*3.14159)
    robot.setAngle(5, sin(t)*0.25*3.14159)
    t+= dt
    rate(100)


<IPython.core.display.Javascript object>

In [2]:
import xml.etree.ElementTree as ET
from vpython import *
import numpy as np
scene = canvas()

# Load URDF
def load_urdf(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    return root

# Extract links and joints
def parse_urdf(root):
    links = {}
    joints = []

    for link in root.findall('link'):
        name = link.attrib['name']
        links[name] = {
            'name': name,
            'visual': link.find('visual')
        }

    for joint in root.findall('joint'):
        joints.append({
            'name': joint.attrib['name'],
            'type': joint.attrib['type'],
            'parent': joint.find('parent').attrib['link'],
            'child': joint.find('child').attrib['link'],
            'axis': joint.find('axis').attrib.get('xyz', '0 0 1'),
            'origin': joint.find('origin').attrib if joint.find('origin') is not None else {'xyz': '0 0 0', 'rpy': '0 0 0'}
        })

    return links, joints

# Convert xyz string to vector
def str_to_vector(s):
    return vector(*map(float, s.strip().split()))

# Create link visual
def create_link_visual(link):
    geom = link['visual'].find('geometry')
    size = vector(0.1, 0.1, 0.1)  # default size
    shape_type = 'box'  # default

    if geom.find('box') is not None:
        shape = geom.find('box')
        size = str_to_vector(shape.attrib['size'])
        shape_type = 'box'

    elif geom.find('cylinder') is not None:
        shape = geom.find('cylinder')
        radius = float(shape.attrib['radius'])
        length = float(shape.attrib['length'])

        # For visualization as a box:
        size = vector(length, 2 * radius, 2 * radius)
        shape_type = 'cylinder'

    # Create the visual object (always as a box)
    return box(size=size, color=color.blue)


# Main VPython structure
class Robot:
    def __init__(self, urdf_path):
        self.links = {}
        self.joints = []
        self.angles = {}

        root = load_urdf(urdf_path)
        link_defs, joint_defs = parse_urdf(root)

        # Create link visuals
        for name, link in link_defs.items():
            self.links[name] = create_link_visual(link)

        # Create joint info
        for joint in joint_defs:
            self.joints.append(joint)
            self.angles[joint['name']] = 0  # initialize angle

    def update(self):
        for joint in self.joints:
            parent = self.links[joint['parent']]
            child = self.links[joint['child']]
            axis_vec = vector(*map(float, joint['axis'].split()))
            origin = joint['origin']
            xyz = str_to_vector(origin.get('xyz', '0 0 0'))
            rpy = list(map(float, origin.get('rpy', '0 0 0').split()))
    
            angle = self.angles[joint['name']]
    
            # Rotate position offset by the joint rotation
            rotated_offset = xyz.rotate(angle=angle, axis=axis_vec)
            child.pos = parent.pos + rotated_offset
    
            # Compute the rotated joint axis
            rotated_axis = axis_vec.rotate(angle=angle)
    
            # 🔄 Set axis and fix size to avoid VPython bug
            saved_size = child.size
            child.axis = rotated_axis.norm() * saved_size.mag  # maintain magnitude
            child.size = saved_size

# Example usage
robot = Robot("arm_urdf.urdf")

def animate():
    t = 0
    while t < 5:
        rate(30)
        t += 0.05
        for joint_name in robot.angles:
            robot.angles[joint_name] = 0.5 * np.sin(t)
        robot.update()

animate()


<IPython.core.display.Javascript object>